In [3]:
import os
import polars as pl
import requests
import time
import plotly.graph_objects as go

from binance.client import Client
from dotenv import load_dotenv

In [4]:
client = Client()
symbol = "BTCUSDT"
interval = Client.KLINE_INTERVAL_1MINUTE
limit = 1000

In [5]:
klines = client.get_klines(
    symbol=symbol,
    interval=interval,
    limit=limit
)

In [11]:
df = pl.DataFrame(klines, schema=[
    ("open_time", pl.Int64),
    ("open", pl.Float64),
    ("high", pl.Float64),
    ("low", pl.Float64),
    ("close", pl.Float64),      
    ("volume", pl.Float64),
    ("close_time", pl.Int64),
    ("quote_asset_volume", pl.Float64),
    ("number_of_trades", pl.Int64),
    ("taker_buy_base_asset_volume", pl.Float64),
    ("taker_buy_quote_asset_volume", pl.Float64),
    ("ignore", pl.Float64),
], orient="row").with_columns([
    pl.from_epoch("open_time", time_unit="ms"),
    pl.from_epoch("close_time", time_unit="ms"),
])

In [7]:
df.null_count()

open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
df.head(5)

open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
datetime[ms],f64,f64,f64,f64,f64,datetime[ms],f64,i64,f64,f64,f64
2025-11-06 08:10:00,103083.9,103119.95,103072.53,103104.62,7.15768,2025-11-06 08:10:59.999,737935.447991,2438,4.17253,430174.21831,0.0
2025-11-06 08:11:00,103104.62,103135.35,103072.05,103072.06,11.56762,2025-11-06 08:11:59.999,1.1928e6,2813,5.03499,519206.829444,0.0
2025-11-06 08:12:00,103072.05,103150.0,103072.05,103150.0,4.53188,2025-11-06 08:12:59.999,467310.025871,1917,3.66613,378030.653243,0.0
2025-11-06 08:13:00,103149.99,103260.01,103149.99,103256.9,14.88666,2025-11-06 08:13:59.999,1.5363e6,3379,12.21164,1.2602e6,0.0
2025-11-06 08:14:00,103256.91,103273.4,103236.97,103273.4,3.92771,2025-11-06 08:14:59.999,405536.521702,1304,3.21192,331623.394179,0.0


In [9]:
dfp = df.to_pandas()

In [10]:
fig = go.Figure(
    data=[
        go.Candlestick(
            x=dfp["open_time"],
            open=dfp["open"],
            high=dfp["high"],
            low=dfp["low"],
            close=dfp["close"],
            increasing_line_color='green',
            decreasing_line_color='red'
        )
    ]
)

fig.show()

In [15]:
client = Client()
data = client.get_aggregate_trades(
    symbol=symbol,
    limit=limit
)

aggtrades = pl.DataFrame(data).rename({
        "a": "agg_trade_id",
        "p": "price",
        "q": "quantity",
        "f": "first_trade_id",
        "l": "last_trade_id",
        "T": "timestamp",
        "m": "is_buyer_maker",
        "M": "is_best_match"
    })

In [16]:
aggtrades.head(5)

agg_trade_id,price,quantity,first_trade_id,last_trade_id,timestamp,is_buyer_maker,is_best_match
i64,str,str,i64,i64,i64,bool,bool
3733524516,"""101531.82000000""","""0.00005000""",5448588589,5448588589,1762477487483,false,true
3733524517,"""101531.92000000""","""0.00197000""",5448588590,5448588590,1762477487483,false,true
3733524518,"""101531.93000000""","""0.00150000""",5448588591,5448588591,1762477487483,false,true
3733524519,"""101532.00000000""","""0.00120000""",5448588592,5448588592,1762477487483,false,true
3733524520,"""101533.32000000""","""0.00005000""",5448588593,5448588593,1762477487495,false,true
